В результате у нас получилось 2 таблички:

Ответы наши сервиса с рекомендациями — в них мы знаем, какому пользователю что мы порекомендовали и в какую группу его отнесли. И, конечно, знаем момент времени, когда это произошло.
Данные о лайках — в них мы знаем, какой пользователь и какой пост лайкнул, в том числе момент времени, когда это произошло.

In [2]:
import pandas as pd

In [4]:
likes = pd.read_csv('../../likes.csv')
views = pd.read_csv('../../views.csv')

In [18]:
views

,user_id,exp_group,recommendations,timestamp
0,128381,control,[3644 4529 4704 5294 4808],1654030803
1,146885,test,[1399 1076 797 7015 5942],1654030811
2,50948,test,[2315 3037 1861 6567 4093],1654030825
3,37703,test,[2842 1949 162 1588 6794],1654030826
4,14661,test,[2395 5881 5648 3417 673],1654030829
...,...,...,...,...
193290,158267,test,[1733 6834 4380 1915 1627],1655240340
193291,63527,control,[2454 191 3873 6404 1588],1655240347
193292,52169,test,[1368 1709 1616 798 5305],1655240354
193293,142402,test,[5895 6984 1978 6548 6106],1655240373


In [5]:
likes

,user_id,post_id,timestamp
0,128381,4704,1654030804
1,146885,1399,1654030816
2,50948,2315,1654030828
3,14661,673,1654030831
4,37703,1588,1654030833
...,...,...,...
230171,31851,5964,1655243535
230172,51512,1498,1655243537
230173,34017,5009,1655243573
230174,13267,1787,1655243692


Теперь давайте попробуем оценить, а улучшаются ли наши метрики в тестовой группе.

Наши данные о показах и кликах хранятся в разных табличках и просто так их не получается сджойнить. Давайте оставим это на потом, а пока попробуем оценить более простыми метриками. Подумайте, какие метрики можно было бы посчитать без джойна показов и кликов.

Соберите табличку, в которой будут пользователи, попавшие в наш эксперимент (то есть те, для которых мы строили хотя бы одну рекомендацию). Посчитайте, кто из них сколько лайков сделал. Наверняка будут пользователи, которые не сделали ни один лайк.

Посчитайте долю пользователей, которая сделала хотя бы один лайк за время эксперимента без разбивки на группы.

In [12]:
views.nunique()

user_id             65013
exp_group               2
recommendations    193295
timestamp          178694
dtype: int64

In [14]:
likes.nunique()

user_id       58171
post_id        6831
timestamp    199397
dtype: int64

А теперь давайте оценим, различаются ли число лайков между группами. Выберите подходящий статистический критерий и проверьте две метрики: доля пользователей с хотя бы одним лайком, число лайков на пользователя. 

Какой статистические критерий наиболее хорошо подходит для метрики "число лайков на пользователя"?

**Mann-Whitney**

Эти две метрики по смыслу довольно похожи и, скорее всего, коррелируют. Так что давайте по ним ответим на вопрос "стали люди ставить больше лайков в тестовой группе". Используем уровень значимости 0.05.

Какие выводы можно сделать про изменение этих двух метрик?

In [39]:
# число лайков на пользователя

from scipy.stats import mannwhitneyu

In [21]:
likes_distr = likes.groupby('user_id')['post_id'].count().reset_index()

In [35]:
likes_table = pd.merge(views, likes, on='user_id', how='left').groupby(['user_id', 'exp_group'])['post_id'].count().reset_index()

In [36]:
likes_table

,user_id,exp_group,post_id
0,200,test,1
1,201,test,18
2,202,test,2
3,212,test,12
4,213,test,35
...,...,...,...
65012,168538,test,15
65013,168541,control,4
65014,168544,test,0
65015,168545,test,20


In [38]:
likes_table.groupby('exp_group')['post_id'].mean()

exp_group
control    15.212771
test       15.750176
Name: post_id, dtype: float64

In [40]:
mannwhitneyu(
    likes_table[likes_table.exp_group == 'control']['post_id'],
    likes_table[likes_table.exp_group == 'control']['post_id'],
)

MannwhitneyuResult(statistic=523390658.0, pvalue=1.0)

p-value > 0.05, а значит, изменения значимы

А теперь давайте сделаем самое сложное — посчитаем hitrate (или долю рекомендаций, в которые пользователи кликнули). Для этого мы хотим собрать в одной табличке информацию о показанных постах и соответствующих кликах.

У нас нет айдишника, по которому мы можем сджойнить, но есть два других способа сделать это:

- По каждому пользователю отсортировать в обратном по времени порядке все лайки и показы рекомендаций (удобнее прямо в одном списке). Далее в цикле пробежаться по этому списку. Если мы встречаем лайк, то запоминаем его (лайков может быть несколько подряд, надо сохранить все). Если мы встречаем показ рекомендаций, то сохраняем его вместе с уже сохранёнными лайками (так как эти лайки следуют за этим показом рекомендаций в "обычном направлении времени"), при этом список сохранёных лайков очищаем (мы уже приписали эти лайки другим показам). Тем самым мы соединим все показы рекомендаций со всеми лайками после них. Нужно только ещё следить за тем, что может быть лайк, который случился через большое время после показа рекомендаций — скорее всего, мы потеряли другой показ рекомендаций, поэтому такой лайк не надо учитывать.

- Сделать join по пользователям всех лайков и всех показов рекомендаций (то есть у нас появятся пары всех показов и всех лайков одного и того же пользователя, но не все эти пары реальны). Далее отфильтровать те ситуации, где лайк предшествовал показу рекомендаций (то есть относился на самом деле к другому показу) или был слишком поздно (тоже относится к другому показу).
Оба эти способа не идеальные и довольно муторные. В pandas проще сделать второй из них. Попробуйте сделать его. В качестве проверки "лайк произошёл слишком поздно после рекомендации" возьмите отсечку 1 час (только лучше сначала просто помечать такие показы, как "отсутствие лайка", потому что если совсем удалять, то можно потерять показ). Добавьте также фильтрацию на то, что в рекомендациях был пост, на который пользователь лайкнул (иначе это точно был другой показ). В самом конце проведите уникализацию всех этих пар в строки показ-лайки (количество лайков может быть нулевым).

На каждом этапе фильтрации следите за размерами таблиц, это поможет вам убедиться, что происходит что-то корректное. Не забудьте учесть тот факт, что могут быть показы рекомендаций без лайков (либо через тип join, либо добавить эти строки в конце). В итоге число строк должно быть похожим на то, которые было сначала в табличке с показами.

Посчитайте долю показов, в которых был хотя бы один лайк.

In [55]:
views['recommendations'] = views['recommendations'].str.split()

In [56]:
views = views.explode('recommendations')

In [57]:
table = pd.merge(views, likes, on='user_id', how='left', suffixes=('_v', '_l'))

In [58]:
table['liked'] = (table.timestamp_l > table.timestamp_v) & ((table.timestamp_l - table.timestamp_v) < 3600)

In [61]:
table[table['post_id'].notna()].groupby('liked').count()

,user_id,exp_group,recommendations,timestamp_v,post_id,timestamp_l
liked,,,,,,
False,3970725,3970725,3970725,3970725,3970725,3970725
True,1192457,1192457,1192457,1192457,1192457,1192457


In [62]:
1192457 / (1192457 + 3970725)

0.23095389626009696

А теперь давайте оценим различие между группами и значимость. z-критерий для долей мы здесь не можем применять, так как у нас в каждой выборке один и тот же пользователь может встречаться несколько раз. Давайте применим бакетный подход (то есть перейдём к бакетам и по ним оценим значимость), чтобы поститать групповой hitrate (или CTR) — доля hitrate по группе/бакету. Используйте 100 бакетов. Уровень значимости останется тем же на уровне 0.05.

Какой вывод мы можем сделать?